In [1]:
import cv2
import numpy as np
import sys


LABELS = ['person','bicycle','car','motorbike','aeroplane','bus','train','truck','boat','traffic light',
          'fire hydrant','stop sign','parking meter','bench','bird','cat','dog','horse','sheep','cow',
          'elephant','bear','zebra','giraffe','backpack','umbrella','handbag','tie','suitcase','frisbee',
          'skis','snowboard','sports ball','kite','baseball bat','baseball glove','skateboard','surfboard',
          'tennis racket','bottle','wine glass','cup','fork','knife','spoon','bowl','banana','apple','sandwich',
          'orange','broccoli','carrot','hot dog','pizza','donut','cake','chair','sofa','pottedplant','bed',
          'diningtable','toilet','tvmonitor','laptop','mouse','remote','keyboard','cell phone','microwave','oven',
          'toaster','sink','refrigerator','book','clock','vase','scissors','teddy bear','hair drier','toothbrush']


COLORS = np.random.uniform(0, 255, size=(len(LABELS), 3))


In [3]:
model = 'C:/Users/ga383983/Car-Distance-Estimation-Indication/yolov2codes/yolov2.weights'
config = 'C:/Users/ga383983/Car-Distance-Estimation-Indication/yolov2codes/yolov2.cfg'
min_confidence=0.14

net = cv2.dnn.readNetFromDarknet(config,model)

In [5]:
import math

# Finction to calculate the distance of object from the camera lense
def dist_calculator(startX,startY,endX,endY,box_width,box_height,img_w,img_h):
    x_3,y_3 = startX, endY - (box_height/7) # top left of the triangle
    #assumption: camera is rasied above the ground so considering 90% of the height of the image height
    x_1,y_1 = img_w/2,0.9*img_h # bottom of the triangle
    x_2,y_2 = endX , endY - (box_height/7) # top right of the triangle

    #find the angle between bottom and right point
    angle_x1_x2 = math.degrees(math.atan2(x_1-x_2,y_1-y_2))
    #find the angle between bottom and left point
    angle_x1_x3 = math.degrees(math.atan2(x_1-x_3,y_1-y_3))

    angle_right = 90 + angle_x1_x2
    angle_left = 90 - angle_x1_x3

    #total angle of view for the car from bottom center point of the image.
    total_angle = angle_right + angle_left

    # Bench length assumed to be 2 metersin millimeters. This value can automated, based on the type of bench used.
    bench_length = 2000.0 
    

    #distance to object = (size of object) x (1°/angular size in degrees) x 57
    #Refer the link for more understadnign on the formula mentioned above - https://www.cfa.harvard.edu/webscope/activities/pdfs/measureSize.PDF
    distance = (bench_length*(1.0/total_angle)*57) / 1000

    print(total_angle)
    print(distance)
    return total_angle,distance

def dist(dist):
    if dist > 2.0: # if distance is greater than 2 meters 
        label = "Far"
    else:
        label = "Near"
    return label


#Loading the image of car
image = cv2.imread('bench_1.jpg')
#print(image.shape)
height, width, ch = image.shape
resize_img = cv2.resize(image,(225,225))


blob = cv2.dnn.blobFromImage(resize_img, 1.0/255.0, (416, 416), True, crop=False)

#blob = cv2.dnn.blobFromImage(resize_img, 0.007843, (300,300), 127.5)

net.setInput(blob)
predictions = net.forward()
probability_index = 5


for i in range(predictions.shape[0]):
    prob_arr=predictions[i][probability_index:]
    class_index = prob_arr.argmax(axis=0)
    confidence = prob_arr[class_index]
    if confidence > min_confidence:
        x_center=predictions[i][0]*width
        y_center=predictions[i][1]*height
        width_box=predictions[i][2]*width
        height_box=predictions[i][3]*height
        
        x1=int(x_center-width_box * 0.5) # Start X coordinate
        y1=int(y_center-height_box * 0.5)# Start Y coordinate
        x2=int(x_center+width_box * 0.5)# End X coordinate
        y2=int(y_center+height_box * 0.5)# End y coordinate
        
        
        cv2.rectangle(image,(x1,y1),(x2,y2), (0,255,120),2)
        
        if class_index== 13 :# 13 is the index of Bench in the LABELS list for prediction

            roi_corners=[[0.0 *width_box, 1.0*height], # left, down
            [x1 + (width_box/5), y2 - (height_box/7)], # left, up
            [x1 + 4*(width_box/5), y2 - (height_box/7)], # right, up
            [1.0 *width,1.0*height]] # right, down

            image = cv2.circle(image, (int(width),int(height)), 3, (0,0,255),-1)
            triangle_pts = [[width/2,0.9*height], # left, down / bottom point of the traingle
                            [x1, y2 - (height_box/7)],# left, up
                            [x2 , y2 - (height_box/7)],# right, up
                            [width/2,0.9*height]]# right, down / / bottom point of the traingle
            src = np.float32(triangle_pts)
            pts = np.array(src, np.int32)
            pts = pts.reshape((-1,1,2))
            
            # drawing the triangle
            cv2.polylines(image,[pts],True,(255,0,0),2)
            
            #distance calculation
            _,distance = dist_calculator(x1,y1,x2,y2,width_box,height_box,width,height)
            
            #label of 'far' or 'near'
            label = dist(distance)
            
            cv2.putText(image,LABELS[class_index]+" "+"{0:.1f}".format(confidence),(x1,y1), cv2.FONT_HERSHEY_SIMPLEX,0.6,(0,0,255),2)
            cv2.putText(image, "Distance: {}".format(round(distance,2)), (x1, y1+13), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 100, 255), 2)
            
            print(label)
        
cv2.imshow("Output", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

49.34901483305525
2.3100765108615673
Far
